In [13]:
import sys
print(sys.version)


3.11.9 (main, Apr  6 2024, 17:59:24) [GCC 11.4.0]


In [14]:
# Define the packages to include

# For delta compatibility with spark, check https://docs.delta.io/latest/releases.html
packages = [
    "org.apache.hadoop:hadoop-aws:3.3.4",  # Hadoop-AWS package,
    "io.delta:delta-spark_2.12:3.2.0" # Delta package
]

In [15]:
# MinIO credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "ouxhmt4Ez73Rt8HSNQFu"
minio_secret_key = "nkEwOH7TsENA7hRSkH0VSRMqdNq65ystE2BODdHW"

In [17]:
from pyspark import SparkConf
conf = (
            SparkConf()
            .setAppName("minIOTest")
            .set("spark.jars.packages", ",".join(packages))
            .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
            .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
            .set("spark.hadoop.fs.s3a.access.key", minio_access_key)
            .set("spark.hadoop.fs.s3a.secret.key", minio_secret_key)
            .set("spark.hadoop.fs.s3a.endpoint", minio_endpoint)
            .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "true")
            .set("spark.hadoop.fs.s3a.path.style.access", "true")
            .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
            .set("spark.hadoop.fs.s3a.attempts.maximum", "1")
            .set("spark.hadoop.fs.s3a.connection.establish.timeout", "5000")
            .set("spark.hadoop.fs.s3a.connection.timeout", "10000")
        )

In [18]:
from pyspark.sql import SparkSession
# Initialize SparkSession

spark =SparkSession.builder.config(conf=conf).master("spark://spark-master:7077").getOrCreate()

In [19]:
df = spark.read.option("header", "true") \
    .csv(f"s3a://test2/DimCountry.csv")
# show data
df.show()

+---------+--------------+
|countryid|       country|
+---------+--------------+
|        1|     Argentina|
|        2|     Australia|
|        3|       Austria|
|        4|    Azerbaijan|
|        5|       Belgium|
|        6|        Brazil|
|        7|      Bulgaria|
|        8|        Canada|
|        9|        Cyprus|
|       10|Czech Republic|
|       11|       Denmark|
|       12|         Egypt|
|       13|       Estonia|
|       14|       Finland|
|       15|        France|
|       16|       Germany|
|       17|        Greece|
|       18|       Hungary|
|       19|         India|
|       20|     Indonesia|
+---------+--------------+
only showing top 20 rows



In [20]:
# Write DataFrame to Delta table in MinIO with overwrite mode
df.write.format("delta").mode("overwrite").save("s3a://test2/DimCountry2")

24/07/16 05:08:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [22]:
# Read Delta table from MinIO
df_delta = spark.read.format("delta").load("s3a://test2/DimCountry2")

# Show the data
df_delta.show()

+---------+--------------+
|countryid|       country|
+---------+--------------+
|        1|     Argentina|
|        2|     Australia|
|        3|       Austria|
|        4|    Azerbaijan|
|        5|       Belgium|
|        6|        Brazil|
|        7|      Bulgaria|
|        8|        Canada|
|        9|        Cyprus|
|       10|Czech Republic|
|       11|       Denmark|
|       12|         Egypt|
|       13|       Estonia|
|       14|       Finland|
|       15|        France|
|       16|       Germany|
|       17|        Greece|
|       18|       Hungary|
|       19|         India|
|       20|     Indonesia|
+---------+--------------+
only showing top 20 rows



In [23]:
spark.stop()